## Contribution per distance/density to overall visits

In [163]:
import numpy as np

def estimate_visitor_distribution(total_visitors, prob_matrix, distances, densities):
    # Calculate the proportion of total visitors from each density based on their relative values
    density_norm_factor = prob_matrix.sum(axis=0)
    density_norm = densities*density_norm_factor
    density_proportions = density_norm/np.sum(density_norm)
    
    # Calculate the total number of visitors attributed to each density
    visitors_per_density = density_proportions * total_visitors

    # Create an empty array to store the estimated visitors for each distance across all densities
    estimated_visitors_by_distance = np.zeros(len(distances))

    # Distribute each density's visitors across distances according to the probability matrix
    for i in range(len(densities)):
        estimated_visitors_by_distance += prob_matrix[:, i] * visitors_per_density[i]

    # Normalize the results to ensure they sum to the total number of visitors
    estimated_visitors_by_distance *= (total_visitors / np.sum(estimated_visitors_by_distance))

    return estimated_visitors_by_distance, visitors_per_density

# Define the probability matrix, distances, and densities
prob_matrix = np.array([
    [0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
    [0.999, 0.979, 0.941, 0.849, 0.610, 0.387, 0.172],
    [0.997, 0.962, 0.899, 0.767, 0.490, 0.288, 0.123],
    [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
    [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.060],
    [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]
])
distances = np.array([0, 1, 2, 3, 4, 5])  # in miles
densities = np.array([1, 5, 10, 20, 50, 100, 250])  # thousands of people within a 5-mile radius

total_visitors = 1000000  # total number of visitors
estimated_visitors, visitors_from_each_density = estimate_visitor_distribution(total_visitors, prob_matrix, distances, densities)

print("Estimated Number of Visitors from Each Distance:")
print(estimated_visitors)
print("\nTotal Estimated Visitors by Distance:", np.sum(estimated_visitors))

# Print total visitors from each density
print("\nTotal Visitors from Each Density:")
for density, visitors in zip(densities, visitors_from_each_density):
    print(f"Density {density}k: {visitors:.0f} visitors")

print("\nTotal Estimated Visitors by Density:", np.sum(visitors_from_each_density))


Estimated Number of Visitors from Each Distance:
[246349.32784016 213723.67207287 180755.30112423 148806.53719399
 118825.60543717  91539.55633158]

Total Estimated Visitors by Distance: 1000000.0

Total Visitors from Each Density:
Density 1k: 9324 visitors
Density 5k: 43427 visitors
Density 10k: 78167 visitors
Density 20k: 128660 visitors
Density 50k: 206686 visitors
Density 100k: 252781 visitors
Density 250k: 280955 visitors

Total Estimated Visitors by Density: 1000000.0000000001


## Scaling expenditure per consumer by square footage of store type

In [164]:
total_weekly_visits = 167 
total_weekly_visits*52

8684

In [165]:
# Constants and data from provided details
total_net_sales = 393247  # in millions of dollars
total_annual_visits = 8684 * 1e6  # converting to total visits per year

# Store-specific square footage and total locations
count_avg_footage = {
    'Supercenters': [3573],
    'Discount Stores': [370],
    'Neighborhood Markets': [799]  # Sum of two Neighborhood Markets data
}

# Average square footage per store type
avg_sqft = {
    'Supercenters': 178000,
    'Discount Stores': 105000,
    'Neighborhood Markets': 42000
}

total_sqft = {
    'Supercenters': 3573*178000,
    'Discount Stores': 370*105000,
    'Neighborhood Markets': 799*42000
}

revenue_per_sq = total_net_sales*1e6 / sum(total_sqft.values())
print(revenue_per_sq)

# Calculate revenue per square foot for each store type
percent_of_sqft = {store_type: (sum(avg_sqft.values())/sqft) for store_type, sqft in avg_sqft.items()}

# Derive scaling factors based on the inverse of average square footage
# Smaller store, higher scaling factor to adjust for less average consumer spending per square foot
scaling_factors = {store_type: min(percent_of_sqft.values()) / sqft for store_type, sqft in percent_of_sqft.items()}

# scaling_factors = {store_type: min(average_sqft.values()) / sqft for store_type, sqft in average_sqft.items()}

# Calculate adjusted average spending per store type using derived scaling factors
average_spending_per_visit = total_net_sales * 1e6 / total_annual_visits  # Average spending per visit
print(average_spending_per_visit)
adjusted_spending = {store_type: average_spending_per_visit * factor for store_type, factor in scaling_factors.items()}

print("Revenue per Square Foot:")
for store, revenue in revenue_per_sqft.items():
    print(f"{store}: ${revenue:.2f}")

print("\nScaling Factors:")
for store, factor in scaling_factors.items():
    print(f"{store}: {factor:.2f}")

print("\nAdjusted Average Spending per Customer per Store Type:")
for store, spending in adjusted_spending.items():
    print(f"{store}: ${spending:.2f} per visit")


555.1184214612607
45.28408567480424
Revenue per Square Foot:
Supercenters: $618.32
Discount Stores: $10122.19
Neighborhood Markets: $11718.43

Scaling Factors:
Supercenters: 1.00
Discount Stores: 0.59
Neighborhood Markets: 0.24

Adjusted Average Spending per Customer per Store Type:
Supercenters: $45.28 per visit
Discount Stores: $26.71 per visit
Neighborhood Markets: $10.69 per visit


In [166]:
class StoreSpendingScaler:
    def __init__(self, total_net_sales, total_annual_visits, square_footage, average_sqft):
        self.total_net_sales = total_net_sales
        self.total_annual_visits = total_annual_visits
        self.square_footage = square_footage
        self.average_sqft = average_sqft

    def calculate_revenue_per_sqft(self):
        revenue_per_sqft = {store_type: (self.total_net_sales * 1e6 / sqft)
                            for store_type, sqft in self.square_footage.items()}
        return revenue_per_sqft

    def derive_scaling_factors(self):
        min_avg_sqft = min(self.average_sqft.values())
        scaling_factors = {store_type: min_avg_sqft / sqft
                           for store_type, sqft in self.average_sqft.items()}
        return scaling_factors

    def calculate_adjusted_spending(self):
        average_spending_per_visit = self.total_net_sales * 1e6 / self.total_annual_visits
        scaling_factors = self.derive_scaling_factors()
        adjusted_spending = {store_type: average_spending_per_visit * factor
                             for store_type, factor in scaling_factors.items()}
        return adjusted_spending

    def summarize_spending_adjustments(self):
        revenue_per_sqft = self.calculate_revenue_per_sqft()
        adjusted_spending = self.calculate_adjusted_spending()

        print("Revenue per Square Foot:")
        for store, revenue in revenue_per_sqft.items():
            print(f"{store}: ${revenue:.2f}")

        print("\nAdjusted Average Spending per Customer per Store Type:")
        for store, spending in adjusted_spending.items():
            print(f"{store}: ${spending:.2f} per visit")

# Example usage
store_data = {
    'total_net_sales': 393247,  # in millions of dollars
    'total_annual_visits': 11960 * 1e6,  # converting to total visits per year
    'square_footage': {
        'Supercenters': 634192948,
        'Discount Stores': 39198914,
        'Neighborhood Markets': 29036713
    },
    'average_sqft': {
        'Supercenters': 178000,
        'Discount Stores': 105000,
        'Neighborhood Markets': 42000
    }
}

scaler = StoreSpendingScaler(**store_data)
scaler.summarize_spending_adjustments()


Revenue per Square Foot:
Supercenters: $620.07
Discount Stores: $10032.09
Neighborhood Markets: $13543.10

Adjusted Average Spending per Customer per Store Type:
Supercenters: $7.76 per visit
Discount Stores: $13.15 per visit
Neighborhood Markets: $32.88 per visit


## Monte Carlo Visitors + Average Spend

In [169]:
def estimate_visitor_distribution_by_store_type(total_visitors, prob_matrix, distances, densities, store_data, num_simulations=1000):
    # Calculate the proportion of total visitors from each density based on their relative values
    # density_norm_factor = prob_matrix.sum(axis=0)
    # density_norm = densities * density_norm_factor
    # density_proportions = density_norm / np.sum(density_norm)

    # Calculate the total number of visitors attributed to each density
    visitor_results=[]
    for _ in range(num_simulations):
        # Randomly adjust probabilities by up to ±10% to simulate changes in demographics or economic conditions
        adjustment_factor = 1 + (np.random.rand(*prob_matrix.shape) - 0.5) * 0.2
        adjusted_matrix = prob_matrix * adjustment_factor
        density_norm_factor = adjusted_matrix.sum(axis=0)
        density_norm = densities * density_norm_factor
        density_proportions = density_norm / np.sum(density_norm)
        
        visitors_per_density = density_proportions * total_visitors

    # Calculate total visitors based on store type and density eligibility
        total_visitors_by_type = {}     
        for store_type, data in store_data.items():
            # Adjust the total visitors according to the store type specifics
            footprint_ratio = data['avg_sqft'] / np.mean([d['avg_sqft'] for d in store_data.values()])
            location_factor = data['locations'] / sum(d['locations'] for d in store_data.values())
    
            # Apply density eligibility
            eligible_visitors_per_density = visitors_per_density * data['density_eligibility']
            eligible_total_visitors = eligible_visitors_per_density.sum()  # Sum of eligible visitors per density
    
            # Calculate the share of visitors for this store type
            raw_share = eligible_total_visitors * footprint_ratio * location_factor
            total_visitors_by_type[store_type] = raw_share
       
    
        # Normalize the visitor distribution to match the total number of visitors
        sum_raw_shares = sum(total_visitors_by_type.values())
        normalization_factor = total_visitors / sum_raw_shares
        for store_type in total_visitors_by_type:
            total_visitors_by_type[store_type] *= normalization_factor
        
        visitor_results.append(list(total_visitors_by_type.values()))

        # Calculate and distribute visitors
        estimated_visitors_by_distance = np.zeros(len(distances))
        for store_type, visitors in total_visitors_by_type.items():
            #visitor_results.append({store_type:visitors})
            for i, is_eligible in enumerate(store_data[store_type]['density_eligibility']):
                if is_eligible:
                    estimated_visitors_by_distance += prob_matrix[:, i] * (eligible_visitors_per_density[i] * normalization_factor)

    return estimated_visitors_by_distance, visitors_per_density, total_visitors_by_type, visitor_results

# Store-specific data with density eligibility
store_data = {
    'Supercenters': {'locations': 3573, 'avg_sqft': 178000, 'density_eligibility': np.array([1, .8, .8, .8, 0.3, 0, 0])},
    'Discount Stores': {'locations': 370, 'avg_sqft': 105000, 'density_eligibility': np.array([0, .2, .2, .2, 0, 0, 0])},
    'Neighborhood Markets': {'locations': 799, 'avg_sqft': 42000, 'density_eligibility': np.array([0, 0, 0, 0, 0.7, 1, 1])}
}

estimated_visitors, visitors_from_each_density, visitors_by_store_type, visitor_results = estimate_visitor_distribution_by_store_type(
    total_visitors=167e6, prob_matrix=prob_matrix, distances=distances, densities=densities, store_data=store_data)

print("Estimated Number of Visitors from Each Distance:")
print(estimated_visitors)
print("\nTotal Estimated Visitors by Distance:", np.sum(estimated_visitors))

# Print total visitors from each density
print("\nTotal Visitors from Each Density:")
for density, visitors in zip(densities, visitors_from_each_density):
    print(f"Density {density}k: {visitors:.0f} visitors")

# Print visitors by store type
print("\nVisitors by Store Type:")
for store_type, visitors in visitors_by_store_type.items():
    print(f"{store_type}: {visitors:.0f} visitors")

# Print visitors by store type
print("\nVisitors by Store Type:")
for store_type, visitors in visitors_by_store_type.items():
    visitors_per_store = visitors / store_data[store_type]['locations']
    print(f"{store_type}: {visitors:.0f} total visitors, {visitors_per_store:.0f} visitors per store")


print("\nTotal Estimated Visitors by Store Type:", sum(visitors_by_store_type.values()))

print("\nTotal Estimated Visitors by Density:", np.sum(visitors_from_each_density)) 
visitor_results = np.array(visitor_results)
#visitor_results[:,1]
# Analyze results
print("Average Estimated Visitors:", np.mean(visitor_results[:,0]),np.mean(visitor_results[:,1]),np.mean(visitor_results[:,2]))
print("Standard Deviation of Visitors:", np.std(visitor_results[:,0]),np.std(visitor_results[:,1]),np.std(visitor_results[:,2]))


Estimated Number of Visitors from Each Distance:
[1.76166335e+08 1.42352893e+08 1.09876028e+08 8.10654538e+07
 5.74282430e+07 3.92335029e+07]

Total Estimated Visitors by Distance: 606122455.6388139

Total Visitors from Each Density:
Density 1k: 1587182 visitors
Density 5k: 7034568 visitors
Density 10k: 13169849 visitors
Density 20k: 20925690 visitors
Density 50k: 34276570 visitors
Density 100k: 43606907 visitors
Density 250k: 46399235 visitors

Visitors by Store Type:
Supercenters: 145779353 visitors
Discount Stores: 1636047 visitors
Neighborhood Markets: 19584601 visitors

Visitors by Store Type:
Supercenters: 145779353 total visitors, 40800 visitors per store
Discount Stores: 1636047 total visitors, 4422 visitors per store
Neighborhood Markets: 19584601 total visitors, 24511 visitors per store

Total Estimated Visitors by Store Type: 167000000.0

Total Estimated Visitors by Density: 166999999.99999997
Average Estimated Visitors: 146092404.37354192 1645147.2620685932 19262448.3643894

In [102]:
import numpy as np

def calculate_revenue_from_visits(visitor_results, avg_spend_base, num_simulations=1000):
    revenue_results = []
    for visitors in visitor_results:
        # Simulate average spending per visitor with some variability
        avg_spend_per_visitor = np.random.normal(avg_spend_base, avg_spend_base * 0.1)  # 10% standard deviation
        revenue = visitors * avg_spend_per_visitor
        revenue_results.append(revenue)
    return revenue_results


prob_matrix = np.array([
    [0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
    [0.999, 0.979, 0.941, 0.849, 0.610, 0.387, 0.172],
    [0.997, 0.962, 0.899, 0.767, 0.490, 0.288, 0.123],
    [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
    [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.060],
    [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]
])
distances = np.array([0, 1, 2, 3, 4, 5])  # in miles
densities = np.array([1, 5, 10, 20, 50, 100, 250])*1000  # thousands of people within a 5-mile radius

# # Example usage
# probability_matrix = np.array([
#     [0.9, 0.8, 0.7],
#     [0.6, 0.5, 0.4],
#     [0.3, 0.2, 0.1]
# ])
# densities = np.array([1000, 2000, 3000])
# avg_spend_base = 50  # Base average spend per visitor




visitor_results = simulate_customer_visits(probability_matrix, densities)
revenue_results = calculate_revenue_from_visits(visitor_results, avg_spend_base)

# Analyze results
print("Average Estimated Visitors:", np.mean(visitor_results))
print("Standard Deviation of Visitors:", np.std(visitor_results))
print("Average Estimated Revenue:", np.mean(revenue_results))
print("Standard Deviation of Revenue:", np.std(revenue_results))


Average Estimated Visitors: 639337.2187199689
Standard Deviation of Visitors: 7725.335836409974
Average Estimated Revenue: 31916716.358955197
Standard Deviation of Revenue: 3281407.26860883


## The Expansion Simulation Class

In [160]:
import numpy as np

class NeighborhoodMarketExpansionModel:
    def __init__(self, demographic_data, market_conditions, financials, investment_factors, historical_data):
        self.demographic_data = demographic_data
        self.market_conditions = market_conditions
        self.financials = financials
        self.investment_factors = investment_factors
        self.historical_data = historical_data

    def calculate_cagr(self, initial_value, final_value, periods):
        if periods <= 0:
            raise ValueError("Periods must be greater than zero for CAGR calculation.")
        return (final_value / initial_value) ** (1 / periods) - 1

    def calculate_cagrs(self):
        revenue_cagr = self.calculate_cagr(
            self.historical_data['initial_revenue'],
            self.historical_data['final_revenue'],
            self.historical_data['periods'])
        cogs_cagr = self.calculate_cagr(
            self.historical_data['initial_cogs'],
            self.historical_data['final_cogs'],
            self.historical_data['periods'])
        ecommerce_cagr = self.calculate_cagr(
            self.historical_data['initial_ecommerce'],
            self.historical_data['final_ecommerce'],
            self.historical_data['periods'])
        return revenue_cagr, cogs_cagr, ecommerce_cagr

    def calculate_cannibalization_effect(self, existing_store_sales, new_store_openings):
        cannibalization_rate = self.market_conditions.get('cannibalization_rate', 0.01)
        adjusted_sales = existing_store_sales * (1 - cannibalization_rate * new_store_openings)
        return adjusted_sales

    def dynamic_ecommerce_growth(self, current_sales):
        ecommerce_growth_factor = self.market_conditions.get('ecommerce_growth_factor', 0.05)
        market_saturation = self.market_conditions.get('market_saturation', 0.1)
        distribution_efficiency = self.market_conditions.get('distribution_efficiency', 0.9)
        adjusted_growth_rate = ecommerce_growth_factor * (1 - market_saturation) * distribution_efficiency
        adjusted_sales = current_sales * (1 + adjusted_growth_rate)
        return adjusted_sales

    def adjust_operational_costs(self, base_cost):
        proximity_factor = self.market_conditions.get('proximity_factor', 0.2)
        distribution_center_proximity = self.market_conditions.get('distribution_center_proximity', 0.5)
        adjusted_cost = base_cost * (1 - proximity_factor * distribution_center_proximity)
        return adjusted_cost

    def labor_investment_needed(self, current_market_share, target_market_share, community_opposition_strength):
        labor_investment_increase = 0
        while current_market_share < target_market_share:
            current_market_share *= (1 + 0.01 * (1 - community_opposition_strength))
            labor_investment_increase += 0.01
        return labor_investment_increase

    def evaluate_labor_investment_impact(self, labor_costs, initial_profit_margin, labor_investment_increase):
        adjusted_labor_costs = labor_costs * (1 + labor_investment_increase)
        adjusted_profit_margin = initial_profit_margin * (1 - 0.05 * labor_investment_increase)
        return adjusted_labor_costs, adjusted_profit_margin

    def run_monte_carlo_simulations(self, num_simulations=1000):
        results = []
        for _ in range(num_simulations):
            current_sales = np.random.normal(100000, 20000)  # Simulated current sales
            new_store_openings = np.random.randint(1, 10)  # Simulated new store openings
            adjusted_sales = self.calculate_cannibalization_effect(current_sales, new_store_openings)
            adjusted_sales = self.dynamic_ecommerce_growth(adjusted_sales)
            adjusted_costs = self.adjust_operational_costs(self.financials['base_operational_cost'])
            net_profit = adjusted_sales - adjusted_costs
            results.append(net_profit)
        average_profit = np.mean(results)
        profit_std_dev = np.std(results)
        return average_profit, profit_std_dev

# Example usage of the class:
model = NeighborhoodMarketExpansionModel(demographic_data={}, market_conditions={}, financials={}, investment_factors={}, historical_data={})
average_profit, profit_std_dev = model.run_monte_carlo_simulations()
print(f"Average Profit from Simulations: {average_profit}")
print(f"Profit Standard Deviation: {profit_std_dev}")


KeyError: 'base_operational_cost'

In [161]:
import numpy as np

class NeighborhoodMarketExpansionModel:
    def __init__(self, demographic_data, market_conditions, financials, investment_factors, historical_data, probability_matrix, densities):
        self.demographic_data = demographic_data
        self.market_conditions = market_conditions
        self.financials = financials
        self.investment_factors = investment_factors
        self.historical_data = historical_data
        self.probability_matrix = probability_matrix
        self.densities = densities

        # Validate the inputs
        self.validate_inputs()

    def validate_inputs(self):
        if not self.probability_matrix.any() or not isinstance(self.probability_matrix, np.ndarray):
            raise ValueError("Probability matrix must be a non-empty numpy array.")
        if not self.densities.any() or not isinstance(self.densities, np.ndarray):
            raise ValueError("Densities must be a non-empty numpy array.")
        if 'avg_spend_per_visitor' not in self.financials or not isinstance(self.financials['avg_spend_per_visitor'], (int, float)):
            raise ValueError("Average spend per visitor must be a numeric value.")
        if 'investment' not in self.financials or not isinstance(self.financials['investment'], (int, float)):
            raise ValueError("Investment must be specified and be a numeric value.")

    def update_market_conditions(self, new_conditions):
        # This method allows for dynamic updates to market conditions
        self.market_conditions.update(new_conditions)

    def simulate_customer_visits(self, num_simulations=1000):
        visitor_results = []
        for _ in range(num_simulations):
            adjustment_factor = 1 + (np.random.rand(*self.probability_matrix.shape) - 0.5) * 0.2
            adjusted_matrix = self.probability_matrix * adjustment_factor
            visitors_per_density = adjusted_matrix.sum(axis=0)
            total_visitors = np.dot(visitors_per_density, self.densities)
            visitor_results.append(total_visitors)
        return visitor_results

    def calculate_revenue_from_visits(self, visitor_results):
        avg_spend_base = self.financials['avg_spend_per_visitor']
        revenue_results = []
        for visitors in visitor_results:
            avg_spend_per_visitor = np.random.normal(avg_spend_base, avg_spend_base * 0.1)
            revenue = visitors * avg_spend_per_visitor
            revenue_results.append(revenue)
        return revenue_results

    def run_monte_carlo_simulations(self):
        visitor_results = self.simulate_customer_visits()
        revenue_results = self.calculate_revenue_from_visits(visitor_results)
        average_revenue = np.mean(revenue_results)
        revenue_std_dev = np.std(revenue_results)
        ebit = average_revenue - self.financials['operational_costs']
        roi = (ebit / self.financials['investment']) * 100  # ROI in percentage
        return {
            "Average Revenue": average_revenue,
            "Revenue Standard Deviation": revenue_std_dev,
            "EBIT": ebit,
            "ROI": roi
        }

# Example usage of the class:
probability_matrix = np.array([
    [0.9, 0.8, 0.7],
    [0.6, 0.5, 0.4],
    [0.3, 0.2, 0.1]
])
densities = np.array([1000, 2000, 3000])

model = NeighborhoodMarketExpansionModel(
    demographic_data={},
    market_conditions={"cannibalization_rate": 0.01, "ecommerce_growth_factor": 0.05},
    financials={"avg_spend_per_visitor": 50, "operational_costs": 20000, "investment": 100000},
    investment_factors={},
    historical_data={},
    probability_matrix=probability_matrix,
    densities=densities
)

# Dynamically updating market conditions
model.update_market_conditions({"cannibalization_rate": 0.02, "ecommerce_growth_factor": 0.07})

results = model.run_monte_carlo_simulations()
print(f"Average Revenue: {results['Average Revenue']}")
print(f"Revenue Standard Deviation: {results['Revenue Standard Deviation']}")
print(f"EBIT: {results['EBIT']}")
print(f"ROI: {results['ROI']}%")


Average Revenue: 419032.20438013243
Revenue Standard Deviation: 42355.16984546945
EBIT: 399032.20438013243
ROI: 399.03220438013244%


## Trying to workout expenses

## Additional insights from appendices

In [ ]:
class WalmartFinancialModel:
    # Existing methods...

    def calculate_supply_chain_efficiencies(self, automation_investments, year):
        # Estimate cost savings and revenue impacts from supply chain automation
        savings = automation_investments * 0.05  # Assuming 5% cost saving from automation
        additional_revenue = savings * 3  # Hypothetical multiplier for revenue from improved delivery speeds
        return savings, additional_revenue
    
    def evaluate_membership_programs(self, subscribers, subscription_fee):
        # Calculate revenue from subscription-based models like Walmart+
        return subscribers * subscription_fee

    def scenario_analysis(self, base_case_parameters, scenarios):
        # Conduct scenario analysis based on varying parameters
        results = {}
        for scenario, params in scenarios.items():
            self.update_market_conditions(params)  # Method to dynamically adjust market conditions
            results[scenario] = self.run_financial_forecast()
        return results


In [ ]:
class SupplyChainModel:
    def __init__(self, initial_investment, cost_savings_rate, revenue_growth_rate):
        self.initial_investment = initial_investment
        self.cost_savings_rate = cost_savings_rate  # Percentage of operational cost savings
        self.revenue_growth_rate = revenue_growth_rate  # Additional revenue growth from enhanced capabilities

    def calculate_cost_savings(self, operational_costs):
        # Calculate cost savings based on the initial investment and operational cost savings rate
        savings = operational_costs * self.cost_savings_rate
        return savings

    def calculate_revenue_growth(self, base_revenue):
        # Calculate additional revenue generated from investments
        additional_revenue = base_revenue * self.revenue_growth_rate
        return additional_revenue

    def simulate_financial_impact(self, operational_costs, base_revenue):
        # Calculate both cost savings and additional revenue
        savings = self.calculate_cost_savings(operational_costs)
        additional_revenue = self.calculate_revenue_growth(base_revenue)
        
        # Net effect on operations
        net_operational_costs = operational_costs - savings
        net_revenue = base_revenue + additional_revenue

        return net_operational_costs, net_revenue

# Example usage
initial_investment = 50000000  # $50 million
operational_costs = 100000000  # $100 million annual operational costs
base_revenue = 500000000  # $500 million base revenue

supply_chain_model = SupplyChainModel(initial_investment, cost_savings_rate=0.1, revenue_growth_rate=0.05)
net_costs, net_revenue = supply_chain_model.simulate_financial_impact(operational_costs, base_revenue)

print(f"Net Operational Costs after Savings: ${net_costs}")
print(f"Net Revenue after Investment: ${net_revenue}")


In [ ]:
class ECommerceGrowth:
    def __init__(self, base_sales, growth_rate, subscribers, subscription_fee, fulfillment_impact):
        self.base_sales = base_sales
        self.growth_rate = growth_rate
        self.subscribers = subscribers
        self.subscription_fee = subscription_fee
        self.fulfillment_impact = fulfillment_impact

    def calculate_incremental_revenue_from_subscriptions(self):
        # Direct revenue from subscriptions
        direct_revenue = self.subscribers * self.subscription_fee
        
        # Indirect revenue could be modeled based on increased purchase frequency
        # Assuming a 5% increase in overall purchases due to membership benefits
        indirect_revenue = self.base_sales * 0.05
        return direct_revenue + indirect_revenue

    def calculate_revenue_from_fulfillment_expansion(self):
        # Additional revenue from enhanced fulfillment capabilities
        # Assuming a 3% increase in sales due to better customer service and faster delivery
        additional_revenue = self.base_sales * self.fulfillment_impact
        return additional_revenue

    def simulate_e_commerce_growth(self):
        # Calculate total e-commerce growth including Walmart+ and fulfillment impacts
        subscription_revenue = self.calculate_incremental_revenue_from_subscriptions()
        fulfillment_revenue = self.calculate_revenue_from_fulfillment_expansion()
        total_revenue = self.base_sales + subscription_revenue + fulfillment_revenue
        return total_revenue

# Example usage
ecommerce_growth = ECommerceGrowth(base_sales=100000000, growth_rate=0.1, subscribers=1000000, subscription_fee=98, fulfillment_impact=0.03)
total_revenue = ecommerce_growth.simulate_e_commerce_growth()

print(f"Total Projected E-Commerce Revenue: ${total_revenue}")


In [ ]:
# Modify the SupplyChainModel to focus exclusively on cost savings
class SupplyChainModel:
    def __init__(self, initial_investment, cost_savings_rate):
        self.initial_investment = initial_investment
        self.cost_savings_rate = cost_savings_rate  # Only focus on cost savings

    def calculate_cost_savings(self, operational_costs):
        # Calculate cost savings based on operational efficiencies
        return operational_costs * self.cost_savings_rate

# Redefine the ECommerceGrowth to explicitly cover all revenue impacts, removing generic revenue growth from supply chain improvements
class ECommerceGrowth:
    def __init__(self, base_sales, subscribers, subscription_fee, fulfillment_impact):
        self.base_sales = base_sales
        self.subscribers = subscribers
        self.subscription_fee = subscription_fee
        self.fulfillment_impact = fulfillment_impact  # Include all revenue enhancements from fulfillment

    def simulate_e_commerce_growth(self):
        # Calculate direct revenue from subscriptions
        direct_revenue = self.subscribers * self.subscription_fee
        # Revenue from enhanced fulfillment capabilities
        additional_revenue = self.base_sales * (0.05 + self.fulfillment_impact)  # Include indirect revenue from subscriptions and fulfillment improvements
        return self.base_sales + direct_revenue + additional_revenue

# This adjustment ensures that the revenue from supply chain improvements is not double-counted with the fulfillment enhancements in ECommerceGrowth.


In [ ]:
import numpy as np

class FinancialModel:
    def __init__(self, base_sales, operational_costs, supply_chain_params, ecommerce_params):
        self.base_sales = base_sales
        self.operational_costs = operational_costs
        self.supply_chain_model = SupplyChainModel(**supply_chain_params)
        self.ecommerce_growth = ECommerceGrowth(**ecommerce_params)

    def run_simulation(self, num_simulations=1000):
        results = []
        for _ in range(num_simulations):
            # Randomly generate input parameters from assumed distributions
            sampled_operational_costs = np.random.normal(self.operational_costs, self.operational_costs * 0.1)  # 10% SD
            sampled_base_sales = np.random.normal(self.base_sales, self.base_sales * 0.1)  # 10% SD

            # Calculate impacts from each model component
            net_operational_costs = self.supply_chain_model.calculate_cost_savings(sampled_operational_costs)
            net_revenue = self.ecommerce_growth.simulate_e_commerce_growth()

            # Calculate net profit or other financial metrics
            net_profit = net_revenue - net_operational_costs
            results.append(net_profit)

        return np.mean(results), np.std(results)  # Return average and standard deviation of results

# Example usage with some hypothetical parameters
params = {
    'base_sales': 500000000,
    'operational_costs': 100000000,
    'supply_chain_params': {'initial_investment': 50000000, 'cost_savings_rate': 0.05},
    'ecommerce_params': {'base_sales': 500000000, 'subscribers': 1000000, 'subscription_fee': 98, 'fulfillment_impact': 0.03}
}
model = FinancialModel(**params)
average_profit, profit_std_dev = model.run_simulation(num_simulations=10000)

print(f"Average Projected Profit: ${average_profit:.2f}")
print(f"Profit Standard Deviation: ${profit_std_dev:.2f}")


In [ ]:
class StoreSpendingScaler:
    def __init__(self, total_net_sales, total_annual_visits, square_footage, average_sqft):
        self.total_net_sales = total_net_sales
        self.total_annual_visits = total_annual_visits
        self.square_footage = square_footage
        self.average_sqft = average_sqft

    def calculate_revenue_per_sqft(self):
        revenue_per_sqft = {store_type: (self.total_net_sales * 1e6 / sqft)
                            for store_type, sqft in self.square_footage.items()}
        return revenue_per_sqft

    def derive_scaling_factors(self):
        min_avg_sqft = min(self.average_sqft.values())
        scaling_factors = {store_type: min_avg_sqft / sqft
                           for store_type, sqft in self.average_sqft.items()}
        return scaling_factors

    def calculate_adjusted_spending(self):
        average_spending_per_visit = self.total_net_sales * 1e6 / self.total_annual_visits
        scaling_factors = self.derive_scaling_factors()
        adjusted_spending = {store_type: average_spending_per_visit * factor
                             for store_type, factor in scaling_factors.items()}
        return adjusted_spending

    def summarize_spending_adjustments(self):
        revenue_per_sqft = self.calculate_revenue_per_sqft()
        adjusted_spending = self.calculate_adjusted_spending()

        print("Revenue per Square Foot:")
        for store, revenue in revenue_per_sqft.items():
            print(f"{store}: ${revenue:.2f}")

        print("\nAdjusted Average Spending per Customer per Store Type:")
        for store, spending in adjusted_spending.items():
            print(f"{store}: ${spending:.2f} per visit")

# Example usage
store_data = {
    'total_net_sales': 393247,  # in millions of dollars
    'total_annual_visits': 11960 * 1e6,  # converting to total visits per year
    'square_footage': {
        'Supercenters': 634192948,
        'Discount Stores': 39198914,
        'Neighborhood Markets': 29036713
    },
    'average_sqft': {
        'Supercenters': 178000,
        'Discount Stores': 105000,
        'Neighborhood Markets': 42000
    }
}

scaler = StoreSpendingScaler(**store_data)
scaler.summarize_spending_adjustments()
